# Consigna: Práctica 6 Ejercicio 3

### Implementación de una CNN para el dataset Iris

Implemente una **Red Neuronal Convolucional (CNN)** para el modelo del **dataset Iris**.  
Pruebe con diferentes configuraciones: variaciones en la cantidad y tamaño de **capas convolucionales**, así como en la estructura de las **capas densas**.  
Registre y compare las métricas. Contraste los resultados con los modelos previos entrenados.

# Resolución

En este ejercicio se implemento una red neuronal convolucional (CNN) utilizando el clásico y conocido dataset Iris, adaptado a un formato de imágenes, lo que es ideal para aplicar redes convolucionales.
El conjunto de datos contiene mediciones de flores pertenecientes a tres especies de Iris (setosa, versicolor y virginica), con el objetivo de predecir correctamente la especie a partir de dichas características.
Para la implemetacion se uso el Framework Keras en cambio de sklearn usado en los anteriores ejercicios, se utiliza este porque es mas apropiado para redes convolucionales.
Para evaluar el desempeño se realizó una división del dataset en 80% para entrenamiento y 20% para prueba, y se calcularon métricas como accuracy y loss que es proporcionado por el Framework utilizado y se van calculando en cada epoca de entrenamiento.

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

2025-11-04 19:41:19.674044: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-04 19:41:19.674407: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-04 19:41:19.733107: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-04 19:41:21.155137: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off,

In [2]:
# Se toma el dataset Iris en imagenes
data_dir = "Datasets/Iris"

# Datos de entrenamiento, 80% de las imagenes
train_data = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=(128, 128),  # tamaño al que se redimensionan las imágenes
    batch_size=16
)

# Datos de validacion, 20% de las imagenes
val_data = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(128, 128),
    batch_size=16
)

Found 421 files belonging to 3 classes.
Using 337 files for training.
Found 421 files belonging to 3 classes.
Using 84 files for validation.


2025-11-04 19:41:22.398269: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [3]:
# Prepocesamiento de datos, primero se normalizan de 0-255(xq son imagenes a color)
# a un normalizado de 0-1.
normalization_layer = tf.keras.layers.Rescaling(1./255)

train_ds = train_data.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_data.map(lambda x, y: (normalization_layer(x), y))

In [4]:
# Distintos filtros como para mejorar el overfitting
data_augmentation = Sequential([
    layers.RandomFlip("horizontal_and_vertical"),   # voltea horizontal y verticalmente
    layers.RandomRotation(0.2),                     # rotación ±20%
    layers.RandomTranslation(0.1, 0.1),             # traslación hasta 10% ancho/alto
    layers.RandomZoom(0.15),                        # zoom ±15%
    layers.RandomBrightness(0.2),                   # brillo ±20%
    layers.RandomContrast(0.2),                     # contraste ±20%
])

In [5]:
# Creacion del modelo convolucional
model = Sequential([
    data_augmentation,
    # Capa convolucional con 32 kernels con cada kernel de 3x3
    Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)), # agarro las imagenes de 128x128 y 3 canales(rgb)
    MaxPooling2D(2,2),
    # Capa convolucional con 64 kernels, se va aprendiendo patrones mas complejos, con cada kernel de 3x3
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    # Flatten convierte la salida de 2D en 1D(vector), necesario para capas densas que agarran un vector como entrada
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(3, activation='softmax') # 3 clases objetivo
])

/home/bruno/Escritorio/uni/2025/2do-cuatri/Inteligencia Artificial/practicos-ia/Practicos-IA/.venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
# Compilacion del modelo con una funcion de activacion, una de perdida y las metricas a utilizar
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',  # se usa esta porque las etiquetas son enteros (0,1,2)
    metrics=['accuracy']
)

In [7]:
# Early stopping para ver si no mejora, cortar el entrenamiento
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
# Ver
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-6)

In [8]:
# Entrenamiento
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=50, # Cantidad de epocas
    callbacks=[early_stop, reduce_lr] # Guardar mejor modelo si va empeorando la perdida en validacion
)

Epoch 1/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 4s 88ms/step - accuracy: 0.5104 - loss: 24.8518 - val_accuracy: 0.6548 - val_loss: 2.2637 - learning_rate: 0.0010
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 78ms/step - accuracy: 0.6142 - loss: 1.1116 - val_accuracy: 0.6548 - val_loss: 0.9345 - learning_rate: 0.0010
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - accuracy: 0.6380 - loss: 1.0122 - val_accuracy: 0.6548 - val_loss: 0.9553 - learning_rate: 0.0010
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - accuracy: 0.6320 - loss: 1.0015 - val_accuracy: 0.6548 - val_loss: 0.9553 - learning_rate: 0.0010
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step - accuracy: 0.6350 - loss: 0.9909 - val_accuracy: 0.6548 - val_loss: 0.9421 - learning_rate: 0.0010
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 79ms/step - accuracy: 0.6350 - loss: 0.9651 - val_accuracy: 0.6548 - val_loss: 0.9660 - learning_rate: 0.0010
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step - accuracy: 0.6320 - loss: 0.9716 - val_ac

In [9]:
# Evaluate calcula la perdida y la metrica accuracy dadas en compilacion
results = model.evaluate(val_data)
print("Resultados:", results)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.6548 - loss: 0.8704
Resultados: [0.8704285621643066, 0.6547619104385376]


#  Conclusión

El modelo alcanzó un accuracy en validación de aproximadamente 0.65, lo que indica que, aunque logra identificar correctamente algunos patrones, presenta un sesgo claro hacia la clase con más ejemplos. Este resultado refleja la importancia de mantener una distribución equilibrada de clases, ya que un dataset desbalanceado tiende a degradar el rendimiento global y reduce la capacidad del modelo para reconocer categorías minoritarias. En definitiva, la calidad de los datos son tan importantes como la arquitectura del modelo.